<h1 style="text-align:center">An introduction to<br />high-performance custom arrays<br /></h1>
<br /><br />
<h4 style="text-align:center; color:rgba(50, 50, 50, 1)">
    <img src="https://avatars2.githubusercontent.com/u/154641?s=460&v=4" style="
        border-bottom-color: rgba(0, 0, 0, 0.0980392);
        border-bottom-left-radius: 300px;
        border-bottom-right-radius: 300px;
        border-bottom-style: solid;
        border-bottom-width: 2px;
        border-image-outset: 0px;
        border-image-repeat: stretch;
        border-image-slice: 100%;
        border-image-source: none;
        border-image-width: 1;
        border-left-color: rgba(0, 0, 0, 0.0980392);
        border-left-style: solid;
        border-left-width: 2px;
        border-right-color: rgba(0, 0, 0, 0.0980392);
        border-right-style: solid;
        border-right-width: 2px;
        border-top-color: rgba(0, 0, 0, 0.0980392);
        border-top-left-radius: 300px;
        border-top-right-radius: 300px;
        border-top-style: solid;
        border-top-width: 2px;
        box-sizing: border-box;
        color: rgb(9, 96, 171);
        cursor: auto;
        display: inline;
        font-family: 'Avenir Next', 'Helvetica Neue', 'Lucida Grande', Helvetica, Arial, sans-serif;
        font-size: 42px;
        font-style: normal;
        font-variant-caps: normal;
        font-weight: normal;
        height: 100px;
        line-height: 95px;
        margin-bottom: 0px;
        margin-left: 0px;
        margin-right: 0px;
        margin-top: 0px;
        outline-color: rgb(9, 96, 171);
        outline-style: none;
        outline-width: 0px;
        padding-bottom: 0px;
        padding-left: 0px;
        padding-right: 0px;
        padding-top: 0px;
        text-align: center;
        text-decoration: none;
        vertical-align: baseline;
        width: 100px;
    " /><br />
Matt Bauman<br />
Julia Computing<br />
</h4>

## The challenge:

Have something that folks from all levels will be able to appreciate:

* **Beginner**
    * Why do we have more than just `Array`s (and `Vector`s and `Matrices`)?
    * What are the advantages of these non-`Array` arrays?
* **Intermediate**
    * Why might _you_ want to subtype (`<:`) an `AbstractArray`?
    * How do you do it and how do the traits work? 
* **Advanced**
    * How do you make it fast?
    * What are the challenges and possible future features?

```
struct CustomArray{T,N,...} <: AbstractArray{T, N}
```

## What is an array?

Container that holds things in a multi-dimensional rectangular shape

* The `AbstractArray` defines the _behaviors_ and _requirements_

## What is an `Array`?

In [ ]:
[1,2,3]

* A built-in subtype of `AbstractArray` that defines a particular _implementation_

## Why would we need more than `Array`?

```julia
for i in 1:4
    # do something four times
end
```

In [ ]:
1:4 isa AbstractArray

In [ ]:
(1:4)[3]

In [ ]:
[1 10 100 1000
 2 20 200 2000
 3 30 300 3000] * (1:4)

## Why would we need more than `Array`?

In [ ]:
function f()
    r = 0.0
    @time for n in 1:2_000_000_000
        r += 1/(n^2)
    end
    r
end
f()

In [ ]:
(2_000_000_000)*sizeof(Int)/(1024*1024*1024)

## Why would we need more than `Array`?

In [ ]:
dump(1:2_000_000_000)

## Ranges

In [ ]:
@which (1:4)[1]

In [ ]:
@which size(1:4)

In [ ]:
@which rand(4, 4) * (1:4)

## Your own custom array: why?

In [ ]:
# Add a default argument
function weighted_sum(A, weights=ones(size(A)))
    r = zero(A[1])
    for i in eachindex(A, weights)
        r += A[i]*weights[i]
    end
    return r
end

In [ ]:
using BenchmarkTools
A = rand(1000,1000)
@btime sum($A)

## What is an array?

_Container that holds things in a rectangular shape_

Julia just needs to know:

* What is its shape? `Base.size`
* How does it get a thing out? `Base.getindex`
* What types of things does it support? And what is its dimensionality?

_That's it._

* How does it change a thing? `Base.setindex!`

## We can make a better default value

In [ ]:
module V1
struct OnesMatrix <: AbstractArray{Int, 2}
    m::Int
    n::Int
end
Base.size(o::OnesMatrix) = (o.m, o.n)
Base.getindex(o::OnesMatrix, i::Int, j::Int) = 1
end

In [ ]:
# Examine x & more (slicing, indexing, multiplication, broadcast)
x = OnesMatrix(1000,1000)
using LinearAlgebra
svdvals(x)

## Our new weighted sum

In [ ]:
function weighted_sum(A, weights=V1.OnesMatrix(size(A)...))
    r = zero(A[1])
    for i in eachindex(A, weights)
        r += A[i]*weights[i]
    end
    return r
end

In [ ]:
@btime weighted_sum($A)

## A closer look at `OnesMatrix`

In [ ]:
x = V1.OnesMatrix(3,3)
x[4, 4]

In [ ]:
x[-123123,123123]

In [ ]:
x[-3] # (and try vector indices)

## Add bounds checking!

In [ ]:
module V2
struct OnesMatrix <: AbstractArray{Int, 2}
    m::Int
    n::Int
end
Base.size(o::OnesMatrix) = (o.m, o.n)
function Base.getindex(o::OnesMatrix, i::Int, j::Int)
    checkbounds(o, i, j)
    return 1
end
end

In [ ]:
V2.OnesMatrix(5,5)[6,6]

## Add bounds checking!

In [ ]:
function weighted_sum(A, weights=V2.OnesMatrix(size(A)...))
    r = zero(A[1])
    @inbounds for i in eachindex(A, weights)
        r += A[i]*weights[i]
    end
    return r
end
@btime weighted_sum($A)

## Opt-in to bounds check removal

In [ ]:
module V3
struct OnesMatrix <: AbstractMatrix{Int}
    m::Int
    n::Int
end
Base.size(o::OnesMatrix) = (o.m, o.n)
@inline function Base.getindex(o::OnesMatrix, i::Int, j::Int)
    @boundscheck begin
        checkbounds(o, i, j)
    end
    1
end
end

In [ ]:
function weighted_sum(A, weights=V3.OnesMatrix(size(A)...))
    r = zero(A[1])
    @inbounds for i in eachindex(A, weights)
        r += A[i]*weights[i]
    end
    return r
end
@btime weighted_sum($A)

## We can do better

In [ ]:
eachindex(V3.OnesMatrix(4, 3))

In [ ]:
eachindex(A) # == 1:1000000

In [ ]:
IndexStyle(V3.OnesMatrix(4, 3), A)

## We can do better

In [ ]:
module V4
struct OnesMatrix <: AbstractArray{Int, 2}
    m::Int
    n::Int
end
Base.size(o::OnesMatrix) = (o.m, o.n)
Base.IndexStyle(::Type{OnesMatrix}) = IndexLinear()
@inline function Base.getindex(o::OnesMatrix, i::Int)
    @boundscheck checkbounds(o, i)
    1
end
end

In [ ]:
function weighted_sum(A, weights=V4.OnesMatrix(size(A)...))
    r = zero(A[1])
    @inbounds for i in eachindex(A, weights)
        r += A[i]*weights[i]
    end
    return r
end
@btime weighted_sum($A)

## Key considerations for bounds-check removal

In [ ]:
x = V4.OnesMatrix(5,5)
@inbounds x[123,123]

* Only occurs within functions — not at global scope

## Key considerations for bounds-check removal

In [ ]:
x = V4.OnesMatrix(5,5)
f(arg) = @inbounds arg[123,123]
f(x)

* Does not work with type-unstabilities

## One last iteration

In [ ]:
weighted_sum([1,2,3,4,5])

In [ ]:
struct OnesArray{N} <: AbstractArray{Int, N}
    dims::NTuple{N, Int}
end
OnesArray(dims...) = OnesArray{length(dims)}(dims)
Base.size(o::OnesArray) = o.dims
Base.IndexStyle(::Type{<:OnesArray}) = IndexLinear()
@inline function Base.getindex(o::OnesArray, i::Int)
    @boundscheck checkbounds(o, i)
    1
end

In [ ]:
# Try out the ones array
OnesArray(3, 3)

## One last iteration

In [ ]:
function weighted_sum(A, weights=OnesArray(size(A)))
    r = zero(A[1])
    @inbounds for i in eachindex(A, weights)
        r += A[i]*weights[i]
    end
    return r
end
@btime weighted_sum($A)

In [ ]:
x = 1:1000000
@btime weighted_sum($x) # check @code_llvm weighted_sum(1:400000)

## The real challenge with `AbstractArray`

* Implementing the optimizations you want!

In [ ]:
x = OnesArray(20,20)
x[[2],[3,4,5]]

In [ ]:
Base.getindex(::OnesArray{2}, I::AbstractVector{Int}, J::AbstractVector{Int}) =
    OnesArray(length(I), length(J))
x[[2],[3,4,5]]

In [ ]:
x[[3,4,5], 2]

In [ ]:
function Base.getindex(o::OnesArray, I...)
    # Handle everything yourself !!

## Broadcasting

```julia
R .= X .+ Y .* Z
```

```julia
for (rᵢ, xᵢ, yᵢ, zᵢ) in ????
    R[rᵢ] = X[xᵢ] + Y[yᵢ]*Z[zᵢ]
end
```

## Broadcast syntax expansion to a run-time representation:

In [ ]:
Meta.@lower 1 .+ 2 .* 3

In [ ]:
import .Broadcast: materialize, broadcasted
_4 = broadcasted(*, 2, 3)
_5 = broadcasted(+, 1, _4)
# materialize(_5)
bc = _5

In [ ]:
# dig into bc
bc.args[2]

## Broadcast implementation trick #1: `@inline`

* `Broadcasted` objects can get huge!
* The trick: just be careful that Julia is able to completely skip creating them in the first place.
* Ensure that everything inlines between `Broadcasted` "creation" and "execution"
* In general purpose code, Julia's `@inline` heuristics are really quite good, but here it pays to `@inline` like crazy

## Easiest way to customize broadcast: `broadcasted`

In [ ]:
OnesArray(3,5) .+ 1.5

In [ ]:
using FillArrays
Base.Broadcast.broadcasted(::typeof(+), o::OnesArray, n::Number) =
    Fill(1+n, size(o))
Base.Broadcast.broadcasted(::typeof(+), n::Number, o::OnesArray) =
    Fill(n+1, size(o))

## Broadcast implementation trick #2: multiple dispatch

* Multiple dispatch is powerful: use it!
* Find places in the API that can benefit from a bit of indirection
* Exploit that indirection to allow specializations to return something that can quack the same
* Here: I introduced the `broadcasted` function instead of calling the `Broadcasted` constructor directly
* This is how ranges behave sanely and efficiently with respect to `+` and `.+` on 0.7: they define a bunch of `broadcasted` methods to opt-out of fusion (when possible) and immediately compute the answer in O(1) time.

## Next-easiest way to customize broadcast: `broadcastable`

* The default `broadcasted` implementation asks each and every argument for its `broadcastable` representation before constructing the `Broadcasted`
* Something is `broadcastable` if it supports `axes` and `getindex`

In [ ]:
using .Broadcast: broadcastable
broadcastable(:sym)

## Broadcast implementation trick #3: wrapper objects

* Don't bend over backwards to support particular things
* Make the things bend for you!
* Note how this compares to traits:
    * Need to support "scalar-like" things like `Type`s (which definitely aren't indexable in the way we want) and non-scalars like arrays
    * Could ask them for a `IsScalar(T)` trait — in which case I'd need to carry that around and eventually ask "should I index into you?" before doing the indexing
    * Or I could just ask them for a common representation and delegate that concern

## The hard way: `BroadcastStyle` traits

In [ ]:
using .Broadcast: BroadcastStyle, AbstractArrayStyle
struct FillStyle{N} <: AbstractArrayStyle{N} end
FillStyle{M}(::Val{N}) where {M,N} = FillStyle{max(N,M)}()
BroadcastStyle(::Type{T}) where {T<:Fill} = FillStyle{ndims(T)}()

In [ ]:
bc = broadcasted(sqrt, Fill(2, 1), Fill(3, 1, 5))

In [ ]:
typeof(bc)

In [ ]:
function copy(bc::Broadcasted{<:FillStyle})
    # implement broadcast on your own

## `BroadcastStyle` promotion

* All arguments are combined with a two-argument `BroadcastStyle` method that "promotes" to a common style
* This is how you can dispatch on one of your types "buried" within a variable number of arguments — you make your `Style` win over other styles



## More details in the interfaces chapter

<br />

#### https://docs.julialang.org/en/stable/manual/interfaces/

Thanks to Jameson Nash, Tim Holy and countless others

## Quick benchmarking tips

* Benchmarks are hard
* Use BenchmarkTools.jl!
* Remember that `@benchmark` and `@btime` simulate the performance of your code as though it were written within a function
* Beware the distinction between global and local references
    * Flag locals with a `$` interpolation
* Microbenchmarks run through the same code and data — can lead to misleading cache and branch performance

In [ ]:
x = OnesArray(5)
i = 1
f_inbounds(x, i) = @inbounds x[i]
@btime $x[$i]
@btime @inbounds $x[$i]
@btime f_inbounds($x, $i)
@btime OnesArray(5)[1]
@btime OnesArray($5)[1]

In [ ]:
@code_llvm f_inbounds(x, 1)

## Views and wrappers

The same concept can be used to layer views and "wrappers" together, for example a `SubArray` (created by `view` or `@views`) _just_ knows how to subset arrays.  That's it:

In [ ]:
X = rand(3,3,3)
V = view(X, [1,1], 2, 1:2)

In [ ]:
V.indices

In [ ]:
V.parent === A

In [ ]:
@code_lowered V[1,2]

## Views and wrappers

This is a common idiom — more and more wrapper arrays are arriving with Julia 0.7/1.0.

In [ ]:
reshape(V, 1, :)

In [ ]:
V'

In [ ]:
reinterpret(Float64, V)